In [1]:
from utils import data_from_url
import os
import pandas as pd

URL_YELLOW_TAXI_201701 = "https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2016-01.csv"
DATA_DIRECTORY = "../data"

### Télécharge les données si non présente en local

In [2]:
data_from_url(URL_YELLOW_TAXI_201701, DATA_DIRECTORY)

File already downloaded


### Chargement des données

In [3]:
name = URL_YELLOW_TAXI_201701.rsplit('/', 1)[-1]
columns_selected = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count',
                    'trip_distance', 'pickup_longitude', 'pickup_latitude',
                    'dropoff_longitude', 'dropoff_latitude', 'total_amount']

dataframe = pd.read_csv(os.path.join(DATA_DIRECTORY, name), usecols=columns_selected)

In [4]:
f"Nombre de lignes : {dataframe.shape[0]}"

'Nombre de lignes : 10906858'

### Filtre les données selon bounding box sur la ville de New York

In [5]:
LATITUDE_BOX = (40.492615, 40.920618)
LONGITUDE_BOX = (-74.256196, -73.676230)

In [6]:
dataframe.columns

Index(['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count',
       'trip_distance', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'total_amount'],
      dtype='object')

In [7]:
df_filter = dataframe[(dataframe.pickup_longitude.between(LONGITUDE_BOX[0], LONGITUDE_BOX[1]) & 
                       dataframe.dropoff_longitude.between(LONGITUDE_BOX[0], LONGITUDE_BOX[1]) &
                       dataframe.pickup_latitude.between(LATITUDE_BOX[0], LATITUDE_BOX[1]) &
                       dataframe.dropoff_latitude.between(LATITUDE_BOX[0], LATITUDE_BOX[1]))]

In [8]:
df_filter.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,total_amount
0,2016-01-01 00:00:00,2016-01-01 00:00:00,2,1.10,-73.990372,40.734695,-73.981842,40.732407,8.8
1,2016-01-01 00:00:00,2016-01-01 00:00:00,5,4.90,-73.980782,40.729912,-73.944473,40.716679,19.3
2,2016-01-01 00:00:00,2016-01-01 00:00:00,1,10.54,-73.984550,40.679565,-73.950272,40.788925,34.3
3,2016-01-01 00:00:00,2016-01-01 00:00:00,1,4.75,-73.993469,40.718990,-73.962242,40.657333,17.3
4,2016-01-01 00:00:00,2016-01-01 00:00:00,3,1.76,-73.960625,40.781330,-73.977264,40.758514,8.8


In [9]:
del dataframe

### On enlève quelques données aberrantes

In [10]:
df_filter.total_amount.quantile([.0001, .9999])

0.0001     -5.80000
0.9999    154.16641
Name: total_amount, dtype: float64

In [11]:
df_filter.trip_distance.quantile([.001, .9999])

0.0010     0.00
0.9999    33.32
Name: trip_distance, dtype: float64

On observe que ...

In [12]:
df_stop = pd.concat([df_filter[['pickup_longitude', 'pickup_latitude']].\
                        rename(columns={"pickup_longitude": "LONGITUDE", "pickup_latitude": "LATITUDE"}), 
                     df_filter[['dropoff_longitude', 'dropoff_latitude']].\
                         rename(columns={"dropoff_longitude": "LONGITUDE", "dropoff_latitude": "LATITUDE"})])

In [13]:
df_stop.max()

LONGITUDE   -73.676285
LATITUDE     40.920601
dtype: float64

In [14]:
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [15]:

from aggofmassivemvtdata.tools_lib import tools_lib
import numpy as np

In [16]:
centroid = df_stop.mean()

In [17]:
# pairs = [
#     ((45.1, 2.3), (50.2, 3.9)),
#     ((32.4, 15.7), (12.5, 84.1)),
# ]

pairs = []
for i, row in df_stop.iterrows():
    pairs.append(((centroid['LATITUDE'], centroid['LONGITUDE']), 
                  (row['LATITUDE'], row['LONGITUDE'])))
    if i == 10000:
        break
distances = np.array(tools_lib.bulk_haversine(pairs))/1000

In [18]:
distances.max()

20.379322732612177

In [19]:
df_stop = df_stop.reset_index().drop('index', axis=1)

In [20]:
from aggofmassivemvtdata.clustering.part2_algo_2 import algo_2

In [21]:
# import logging
# logging.basicConfig(level=logging.INFO)

In [22]:
grille = algo_2(df_stop[['LATITUDE', 'LONGITUDE']].to_numpy(), 100, redistribute_point=False)


KeyboardInterrupt: 

In [ ]:
centroids = grille.getAllCentroids()

In [ ]:
f"Nombre de centroids : {centroids.shape[0]}"